In [ ]:
!pip install hummingbird_ml

In [1]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import load_breast_cancer
from hummingbird.ml import convert

#### Create and fit the model

In [2]:
# Create and train a RandomForestClassifier model
X, y = load_breast_cancer(return_X_y=True)
skl_model = RandomForestClassifier(n_estimators=500, max_depth=7)
skl_model.fit(X, y)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=7, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=500,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

#### Time scikit-learn

In [3]:
%%timeit
pred = skl_model.predict(X)

42.8 ms ± 473 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


#### Convert SKL model to PyTorch

In [4]:
model = convert(skl_model, 'torch')

#### Time PyTorch - CPU

In [5]:
%%timeit
pred_cpu_hb = model.predict(X)

18.3 ms ± 289 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


#### Switch PyTorch from CPU to GPU

In [6]:
%%capture 
model.to('cuda')

#### Time PyTorch - GPU

In [7]:
%%timeit
pred_gpu_hb = model.predict(X)

1.21 ms ± 2.16 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
